In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Lobby as LobbyModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel
from app.models import LobbySeed as LobbySeedModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)
import sqlite3
import pandas as pd
import uuid
import requests
import json
import random


In [3]:
db.session.close()

In [4]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [5]:
# purge all tables first

Models = [
    BracketModel, 
    LobbyModel,
    TournamentModel, 
    MatchModel, 
    RoundModel, 
    UserModel, 
    LobbySeedModel
]

for Model in Models:
    purge(Model)
db.session.commit()

In [6]:
# Create the connection
cnx = sqlite3.connect(r'./app.db')

In [7]:
# register user like
# curl -i -X POST -H "Content-Type: application/json" -d '{"username":"miguel","password":"python"}' http://127.0.0.1:5000/api/users

In [8]:
url = 'http://127.0.0.1:5000/api/users'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "username":"miguel",
    "password":"python",
    "email":"miguel@gmail.com"
})

r = requests.post(url, data=payload, headers=headers)

In [9]:
# log miguel in
url = 'http://127.0.0.1:5000/api/login'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"Success":"miguel logged in"}\n'

In [10]:
# log miguel out
url = 'http://127.0.0.1:5000/api/logout'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username":"miguel",
#     "password":"python",
#     "email":"miguel@gmail.com"
# })

r = requests.get(url, headers=headers, auth=('miguel', 'python'))
r.content

b'{"Success":"logged out"}\n'

In [11]:
url = 'http://127.0.0.1:5000/api/users'
# post the rest of the users
users = [
    ('TPN', 'TPN@example.com'),
    ('AngryFalco', 'AngryFalco@example.com'),
    ('Sunrisebanana', 'Sunrisebanana@example.com'),
    ('Ptolemy', 'Ptolemy@example.com'),
    ('Vik', 'Vik@example.com'),
    ('Kevin', 'Kevin@example.com'),
    ('Spaceghost', 'Spaceghost@example.com'),
    ('Burnaby', 'Burnaby@example.com'),
]

for username, email in users:
    payload = json.dumps({
        "username":username,
        "password":username.lower(),
        "email":email.lower()
    })
    r = requests.post(url, data=payload, headers=headers)


In [12]:
# or get a token
# i.e. curl -u miguel:python -i -X GET http://127.0.0.1:5000/api/token

In [13]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4Mzg3MDIxNSwiZXhwIjoxNTgzODcwODE1fQ.eyJpZCI6MX0.xPLYtxiMivF72v70O7-x2IUruRrdC_iWY8huaQE7gRQascnYdtpGszqMt4B_vdCxQj4GUTi9Za2F6Odf1f-MuA'

In [14]:
# now this user can login with a token
url = 'http://127.0.0.1:5000/api/login'

r = requests.get(url, auth=(tkn, 'unused'))

In [15]:
# miguel creates lobby

url = 'http://127.0.0.1:5000/api/create/lobby/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({
    "tournament_name" : "miguel's tourneys"
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
json.loads(r.content)

{'Lobby Created': "Lobby created for tournament: miguel's tourneys"}

# Empty lobby 

In [16]:
q = \
"""
    select * from lobby_seeds where lobby_id == 1
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,user_id,lobby_id,seed


In [17]:
q = \
"""
    select * from lobby
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,tournament_name,tournament_id,to_id
0,1,miguel's tourneys,None,1


In [18]:
q = \
"""
    select * from user
"""

# posted a user
df = pd.read_sql_query(q, cnx)
df

,id,role,username,email,password_hash,about_me,last_seen
0,1,User,miguel,miguel@gmail.com,pbkdf2:sha256:150000$NwGLKJDp$18e43b77f747a1ab...,None,2020-03-10 19:56:54.580192
1,2,User,TPN,tpn@example.com,pbkdf2:sha256:150000$UbjEvSea$5cc2129ebab00c34...,None,2020-03-10 19:56:54.876640
2,3,User,AngryFalco,angryfalco@example.com,pbkdf2:sha256:150000$tbLT88gs$a3bf765a45a77e2a...,None,2020-03-10 19:56:54.960640
3,4,User,Sunrisebanana,sunrisebanana@example.com,pbkdf2:sha256:150000$5Ai7JJaU$b3ebb9b92f877c07...,None,2020-03-10 19:56:55.041048
4,5,User,Ptolemy,ptolemy@example.com,pbkdf2:sha256:150000$E0X6nF90$47901789d6227297...,None,2020-03-10 19:56:55.121539
5,6,User,Vik,vik@example.com,pbkdf2:sha256:150000$S05Qgyow$f1d7cef53c47f416...,None,2020-03-10 19:56:55.203662
6,7,User,Kevin,kevin@example.com,pbkdf2:sha256:150000$WrZxarTI$65789a28dca31d67...,None,2020-03-10 19:56:55.284688
7,8,User,Spaceghost,spaceghost@example.com,pbkdf2:sha256:150000$V1eg1Dsn$fb6dac3194ddb3c1...,None,2020-03-10 19:56:55.366591
8,9,User,Burnaby,burnaby@example.com,pbkdf2:sha256:150000$jZoGIUx7$564b90116a775a1b...,None,2020-03-10 19:56:55.447110


In [19]:
url = 'http://127.0.0.1:5000/api/token'
r = requests.get(url, auth=('miguel', 'python'))
tkn = json.loads(r.content)['token']
tkn

'eyJhbGciOiJIUzUxMiIsImlhdCI6MTU4Mzg3MDIxNSwiZXhwIjoxNTgzODcwODE1fQ.eyJpZCI6MX0.xPLYtxiMivF72v70O7-x2IUruRrdC_iWY8huaQE7gRQascnYdtpGszqMt4B_vdCxQj4GUTi9Za2F6Odf1f-MuA'

In [20]:
users = df.username.values
np.random.shuffle(users)
import random

# miguel adds users to his lobby

url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

seeds = random.sample(range(1, len(users)+1), len(users)) 
roles = ["User" for u in users]
for u, s, r in zip(users, roles, seeds):
    print(f'adding {u}, {s}, {r}')
    payload = json.dumps({
        "username" : u,
        "role" : s,
        "seed" : r
    })
    r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
    print(json.loads(r.content))
    

adding Sunrisebanana, User, 6
{'Added': 'User Sunrisebanana to lobby 1'}
adding miguel, User, 9
{'Added': 'User miguel to lobby 1'}
adding AngryFalco, User, 8
{'Added': 'User AngryFalco to lobby 1'}
adding Kevin, User, 7
{'Added': 'User Kevin to lobby 1'}
adding Spaceghost, User, 1
{'Added': 'User Spaceghost to lobby 1'}
adding Vik, User, 3
{'Added': 'User Vik to lobby 1'}
adding Burnaby, User, 5
{'Added': 'User Burnaby to lobby 1'}
adding Ptolemy, User, 2
{'Added': 'User Ptolemy to lobby 1'}
adding TPN, User, 4
{'Added': 'User TPN to lobby 1'}


In [21]:
url = 'http://127.0.0.1:5000/api/lobby/1'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username" : "AngryFalco",
#     "role" : "User"
# })
r = requests.get(url, headers=headers,)
json.loads(r.content)

{'id': 1, 'tournament_name': "miguel's tourneys"}

In [22]:
# miguel adds a guest user

url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps({
    "username" : "guestb",
    "role" : "Guest",
    "seed" : 10
})
r = requests.post(url, data=payload, headers=headers, auth=(tkn, 'unused'))
print(json.loads(r.content))

{'Added': 'Guest guestb to lobby 1'}


In [23]:
# log miguel out
url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

# TPN tries to add his own guests to miguel's lobby
url = 'http://127.0.0.1:5000/api/lobby/1/add-user/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps({
    "username" : "guestx",
    "role" : "Guest",
    "seed" : 1
})
r = requests.post(url, data=payload, headers=headers, auth=('TPN', 'tpn'))
print(json.loads(r.content))

UserModel \
    .query \
    .filter_by(username='guesta', role='Guest') \
    .first() is None

{'Unauthorized': 'TPN cannot add entrants to this lobby'}


True

In [24]:
url = 'http://127.0.0.1:5000/api/lobby/1/entrants'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
# payload = json.dumps({
#     "username" : "AngryFalco",
#     "role" : "User"
# })
r = requests.get(url, headers=headers,)
content =  json.loads(r.content)
content

[{'lobby': 1, 'user': 4, 'id': 1, 'seed': 6},
 {'lobby': 1, 'user': 1, 'id': 2, 'seed': 9},
 {'lobby': 1, 'user': 3, 'id': 3, 'seed': 8},
 {'lobby': 1, 'user': 7, 'id': 4, 'seed': 7},
 {'lobby': 1, 'user': 8, 'id': 5, 'seed': 1},
 {'lobby': 1, 'user': 6, 'id': 6, 'seed': 3},
 {'lobby': 1, 'user': 9, 'id': 7, 'seed': 5},
 {'lobby': 1, 'user': 5, 'id': 8, 'seed': 2},
 {'lobby': 1, 'user': 2, 'id': 9, 'seed': 4},
 {'lobby': 1, 'user': 10, 'id': 10, 'seed': 10}]

In [25]:
pd.read_sql_query(f'select * from lobby', cnx)

,id,tournament_name,tournament_id,to_id
0,1,miguel's tourneys,None,1


In [26]:
pd.read_sql_query(f'select * from lobby_seeds', cnx)

,id,user_id,lobby_id,seed
0,1,4,1,6
1,2,1,1,9
2,3,3,1,8
3,4,7,1,7
4,5,8,1,1
5,6,6,1,3
6,7,9,1,5
7,8,5,1,2
8,9,2,1,4
9,10,10,1,10


In [27]:
url = 'http://127.0.0.1:5000/api/lobby/1/create-tournament/'
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
payload = json.dumps({})
r = requests.post(url, payload, headers=headers, auth=(tkn, 'unused'))
# r = requests.post(url, payload, headers=headers, auth=('TPN', 'tpn'))
r



<Response [200]>

In [28]:
r.content

b'{"Success":"Created tournament 1"}\n'

In [29]:
json.loads(r.content)

{'Success': 'Created tournament 1'}

In [30]:
# create the dataframe from a query

print('#'*68)
print()

print('Users\' table')
df = pd.read_sql_query(q, cnx)
# df

['tournament', 'bracket', 'bracket_entrants', 'round', 'match', 'lobby', 
'lobby_seeds']
    
pd.read_sql_query(f'select * from tournament', cnx)

####################################################################

Users' table


,id,n_entrants,name,organizer_id,finished
0,1,10,miguel's tourneys,1,0


In [31]:
pd.read_sql_query(f'select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [32]:
pd.read_sql_query(f'select * from bracket_entrants', cnx)

,user_id,bracket_id
0,8,1
1,5,1
2,6,1
3,2,1
4,9,1
5,4,1
6,7,1
7,3,1
8,1,1
9,10,1


In [33]:
pd.read_sql_query(f'select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [34]:
pd.read_sql_query(f'select * from match', cnx)

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,0f2d2f07-f213-4a25-951b-060e86ca5a14,8.0,None,3.0,None,None,1,5.0,10.0
1,2,c6d7fb14-2d98-4740-b949-a509dbd01755,5.0,None,7.0,None,None,1,5.0,10.0
2,3,8133c2ed-940f-4fb8-839b-286aabab15b4,6.0,None,4.0,None,None,1,6.0,11.0
3,4,fb913a08-ca10-4eac-b58e-c9e37ef83da4,2.0,None,9.0,None,None,1,6.0,11.0
4,5,77780c3a-c819-41e0-b4a9-659cb83668a0,NaN,None,NaN,None,None,2,7.0,12.0
5,6,8ef31752-0f0c-415e-b4ad-208818d61c69,NaN,None,NaN,None,None,2,7.0,13.0
6,7,1d3824a6-6f96-4897-81d1-cf6f30baad61,NaN,None,NaN,None,None,3,8.0,15.0
7,8,d7dce989-52e3-413e-ad07-c815a242a331,NaN,None,NaN,None,None,4,9.0,9.0
8,9,f2b708bc-6b49-4015-944d-6230b5066582,NaN,None,NaN,None,None,5,NaN,NaN
9,10,b4838fe2-ee89-403a-afd3-5569c050738b,NaN,None,NaN,None,None,6,12.0,NaN


In [35]:
pd.read_sql_query(f'select * from lobby', cnx)

,id,tournament_name,tournament_id,to_id


In [36]:
pd.read_sql_query(f'select * from lobby_seeds', cnx)

,id,user_id,lobby_id,seed
